In [1]:
import math
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os

In [11]:
# Set random seed
torch.manual_seed(42)

# Load data
df = pd.read_csv('./DataProcess/station/1037A/1037A_2020.csv')
folderPath = './DataProcess/station/1037A'
df.dropna()
df.head()
features = ['PM10', 'SO2', 'NO2', 'O3', 'CO']
target = 'PM2.5'
print(df.head())
print(type(df['date']))

       date  hour   AQI  PM2.5   PM10   SO2   NO2   O3   CO
0  20200101     0  74.0   54.0   77.0  24.0  57.0  9.0  1.5
1  20200101     1  70.0   51.0   84.0  21.0  56.0  8.0  1.3
2  20200101     2  75.0   55.0   97.0  24.0  58.0  8.0  1.5
3  20200101     3  82.0   60.0  107.0  29.0  62.0  9.0  1.6
4  20200101     4  80.0   59.0  109.0  26.0  60.0  9.0  1.5


In [10]:
# 将日期和小时列合并为一个字符串列
df['datetime'] = df['date'].astype(str).str.zfill(8) + '-' + df['hour'].astype(str).str.zfill(2)
# 将合并后的字符串列转换为datetime类型
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y%m%d-%H')
print(df)

ValueError: time data '000.0202-00' does not match format '%Y%m%d-%H' (match)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')